# Demonstration of mip_cmor_tables code
This isn't finished, but is borderline functional as a library to introduce changes such as new variables

In [1]:
import mip_cmor_tables


### Load the tables collection

In [2]:
# all checksums validated on load, change optional keyword to False if any manual changes are made
cmor_tables = mip_cmor_tables.MIPTableCollection('../Tables', validate_checksums=True)
print(cmor_tables.location)

../Tables


Access the generic CV and non variable definition information dictionaries directly (or through .data attribute)

In [3]:
# Key components directly available
print(cmor_tables.generic_cv['Header'])
print(cmor_tables.coordinate.data['Header'])
print(cmor_tables.formula_terms['Header'])
print(cmor_tables.grids.data['Header'])

{'CV_collection_modified': '2022-09-05', 'CV_collection_version': '6.3.0.0', 'author': 'Matt Mizielinski <matthew.mizielinski@metoffice.gov.uk>', 'checksum': 'md5: b7696cc5fa72537af51f3b37e3e70e37', 'institution_id': 'MOHC', 'previous_commit': 'To be added', 'specs_doc': 'v6.3.0 (link TBC)'}
{'checksum': 'md5: 075c8144e4b35d140242b028ed3c0004'}
{'checksum': 'md5: 58603f544c5db5e9d16e631970aabf92'}
{'Conventions': 'CF-1.7 CMIP-6.3', 'approx_interval': '<NEEDS WORK>', 'checksum': 'md5: 39e7a038c812a390fc0e412653363015', 'cmor_version': '4.0', 'data_specs_version': '6.3.0.0', 'generic_levels': '', 'int_missing_value': '-999', 'missing_value': '1e20', 'product': 'model-output', 'table_date': '2022-09-05', 'table_id': 'grids'}


## Get hold of a table, and obtain a variable

In [4]:
# Retrieve a table (just an extended dictionary with ability to validate checksums)
APmon = cmor_tables.tables['APmon']
APmon.validate_checksum()
APmon['variable_entry']['tas']



{'branded_variable_name': 'tas_mon-ap-tav-h02-hxy',
 'cell_measures': 'area: areacella',
 'cell_methods': 'area: time: mean',
 'comment': 'near-surface (usually, 2 meter) air temperature',
 'dimensions': ['longitude', 'latitude', 'time', 'height2m'],
 'frequency': 'mon',
 'long_name': 'Near-Surface Air Temperature',
 'modeling_realm': ['atmos'],
 'out_name': 'tas',
 'positive': '',
 'provenance': {'CMIP6': {'dreq_uid': 'bab9237c-e5dd-11e5-8482-ac72891c3257',
   'mip_table': 'Amon',
   'variable_name': 'tas'}},
 'standard_name': 'air_temperature',
 'type': 'real',
 'units': 'K',
 'validation': {'ok_max_mean_abs': 295.0,
  'ok_min_mean_abs': 255.0,
  'valid_max': 350.0,
  'valid_min': 170.0}}

#### modify dictionary and confirm that checksum no longer match (leading to exception being raised)

In [5]:
# update an element of the dictionary and confirm that checksums no longer match
original_value = APmon['Header']['approx_interval']
APmon['Header']['approx_interval'] = 'junk'
APmon.validate_checksum()

RuntimeError: Filename            "../Tables/APmon.json"
Expected checksum   "md5: 41fceb0479b84a65ecf89be89da06def"
Calculated checksum "md5: 85ad89519c5a644af70a4ff42c38cb1f"

#### Update checksums and all validates fine

In [6]:
# update the checksum and everything should validate again
APmon.update_checksum()
APmon.validate_checksum()

In [7]:
# restore original value
APmon['Header']['approx_interval'] = original_value
APmon.update_checksum()

#### Obtain variable directly from MIPTableCollection as a MIPVariable object (relatively simple dataclass)

In [8]:
APmon_tas = cmor_tables.get_variable('APmon', 'tas')

In [9]:
print(APmon_tas)
vars(APmon_tas)


MIPVariable(parent=<mip_cmor_tables.MIPTableCollection object at 0x7f82797afb50>, branded_variable_name='tas_mon-ap-tav-h02-hxy', cell_measures='area: areacella', cell_methods='area: time: mean', comment='near-surface (usually, 2 meter) air temperature', dimensions=['longitude', 'latitude', 'time', 'height2m'], frequency='mon', long_name='Near-Surface Air Temperature', modeling_realm=['atmos'], name='tas', out_name='tas', positive='', provenance={'CMIP6': {'dreq_uid': 'bab9237c-e5dd-11e5-8482-ac72891c3257', 'mip_table': 'Amon', 'variable_name': 'tas'}}, standard_name='air_temperature', table_id='APmon', type='real', units='K', validation={'ok_max_mean_abs': 295.0, 'ok_min_mean_abs': 255.0, 'valid_max': 350.0, 'valid_min': 170.0}, flag_meanings=None, flag_values=None)


{'parent': <mip_cmor_tables.MIPTableCollection at 0x7f82797afb50>,
 'branded_variable_name': 'tas_mon-ap-tav-h02-hxy',
 'cell_measures': 'area: areacella',
 'cell_methods': 'area: time: mean',
 'comment': 'near-surface (usually, 2 meter) air temperature',
 'dimensions': ['longitude', 'latitude', 'time', 'height2m'],
 'frequency': 'mon',
 'long_name': 'Near-Surface Air Temperature',
 'modeling_realm': ['atmos'],
 'name': 'tas',
 'out_name': 'tas',
 'positive': '',
 'provenance': {'CMIP6': {'dreq_uid': 'bab9237c-e5dd-11e5-8482-ac72891c3257',
   'mip_table': 'Amon',
   'variable_name': 'tas'}},
 'standard_name': 'air_temperature',
 'table_id': 'APmon',
 'type': 'real',
 'units': 'K',
 'validation': {'ok_max_mean_abs': 295.0,
  'ok_min_mean_abs': 255.0,
  'valid_max': 350.0,
  'valid_min': 170.0},
 'flag_meanings': None,
 'flag_values': None}

#### Construct a new variable linked to the cmor_tables (needs link to ensure that the coordinates information and the like can be validated)

In [10]:
new_variable = mip_cmor_tables.MIPVariable(cmor_tables)
vars(new_variable)

{'parent': <mip_cmor_tables.MIPTableCollection at 0x7f82797afb50>,
 'branded_variable_name': None,
 'cell_measures': None,
 'cell_methods': None,
 'comment': None,
 'dimensions': [],
 'frequency': None,
 'long_name': None,
 'modeling_realm': None,
 'name': None,
 'out_name': None,
 'positive': None,
 'provenance': {},
 'standard_name': None,
 'table_id': None,
 'type': None,
 'units': None,
 'validation': {},
 'flag_meanings': None,
 'flag_values': None}

#### provide invalid data and check

In [11]:
new_variable.dimensions = ['junk']
new_variable.check()

AssertionError: Dimensions "junk" not found in the coordinates.json file

#### provide valid data and check

In [12]:
new_variable.dimensions = ['latitude', 'longitude', 'time']
new_variable.check()

#### Add a new variable by copying  APmon/tas

In [13]:
from copy import deepcopy

In [14]:
# Copy APmon_tas and use it to create a climatological equivalent
APmonClim_tasClim = deepcopy(APmon_tas)
APmonClim_tasClim.dimensions[2] = 'time2'
APmonClim_tasClim.cell_methods = 'area: mean time: mean within years time: mean over years'
APmonClim_tasClim.frequency = 'monC'
APmonClim_tasClim.out_name = 'tasClim'
APmonClim_tasClim.name = 'tasClim'
APmonClim_tasClim.branded_variable_name = APmonClim_tasClim.branded_variable_name.replace('tas', 'tasClim').replace('mon', 'monC')
APmonClim_tasClim.provenance = {'Notes': 'New variable introduced for testing'}

In [15]:
# Check it for consistency
APmonClim_tasClim.check()

In [16]:
# Add entry to table APmonClim (needs api)
cmor_tables.tables['APmonClim']['variable_entry']['tasClim'] = APmonClim_tasClim.to_json()

In [17]:
# Retrieve new variable from mip table
cmor_tables.get_variable('APmonClim', 'tasClim')

MIPVariable(parent=<mip_cmor_tables.MIPTableCollection object at 0x7f82797afb50>, branded_variable_name='tasClim_monC-ap-tav-h02-hxy', cell_measures='area: areacella', cell_methods='area: mean time: mean within years time: mean over years', comment='near-surface (usually, 2 meter) air temperature', dimensions=['longitude', 'latitude', 'time2', 'height2m'], frequency='monC', long_name='Near-Surface Air Temperature', modeling_realm=['atmos'], name='tasClim', out_name='tasClim', positive='', provenance={'Notes': 'New variable introduced for testing'}, standard_name='air_temperature', table_id='APmonClim', type='real', units='K', validation={'ok_max_mean_abs': 295.0, 'ok_min_mean_abs': 255.0, 'valid_max': 350.0, 'valid_min': 170.0}, flag_meanings=None, flag_values=None)

In [18]:
# Save to disk
cmor_tables.save()

In [19]:
# Show diff
!git diff ../Tables/APmonClim.json

diff --git a/Tables/APmonClim.json b/Tables/APmonClim.json
index 8ad5611..848ea42 100644
--- a/Tables/APmonClim.json
+++ b/Tables/APmonClim.json
@@ -2,7 +2,7 @@
   "Header": {
     "Conventions": "CF-1.7 CMIP-6.3",
     "approx_interval": "<NEEDS WORK>",
-    "checksum": "md5: 6b1dc8e2cad0c8e9115d9e9c855205b5",
+    "checksum": "md5: 9d937c8817b27a11a92d5b75ac854057",
     "cmor_version": "4.0",
     "data_specs_version": "6.3.0.0",
     "generic_levels": "",
@@ -253,6 +253,37 @@
         "valid_max": "",
         "valid_min": ""
       }
+    },
+    "tasClim": {
+      "branded_variable_name": "tasClim_monC-ap-tav-h02-hxy",
+      "cell_measures": "area: areacella",
+      "cell_methods": "area: mean time: mean within years time: mean over years",
+      "comment": "near-surface (usually, 2 meter) air temperature",
+      "dimensions": [
+        "longitude",
+        "latitude",
+        "time2",
+        "height2m"
+      ],
+      "frequency": "monC",
+      "long_name": "Near-Sur